# Predicting Salaries & Web Scraping
## Scraping Indeed.com

In [1]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [2]:
from wordcloud import (WordCloud, get_single_color_func)
import matplotlib.pyplot as plt
import numpy as np
import pickle as pk




Simple Class to output a nice graphic with words, where size of words = occurances

In [3]:
class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)

In [4]:
def pltbarangcantik(df):
    text = ''
    for kerja in df['jobs'].unique():
        text += (str(kerja) + ' ') * int(df.importance[kerja] * 10000)

    positives = [str(index) for index in df.head(20).index if df.impact_coef[index]>0]
    negatives = [str(index) for index in df.head(20).index if df.impact_coef[index]<0]

    # Since the text is small collocations are turned off and text is lower-cased
    wc = WordCloud(background_color="white", collocations=False).generate(text.lower())

    color_to_words = {'blue': positives, 'brown': negatives}

    default_color = 'black'

    # Create a color function with single tone
    grouped_color_func = SimpleGroupedColorFunc(color_to_words, default_color)

    # Apply our color function
    wc.recolor(color_func=grouped_color_func)

    # Plot
    plt.figure(figsize=(12,10))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()


Perhaps a better scraper than Irvin's

In [5]:
Countries = {"SG":"Singapore","US":"United States","MY":"Malaysia","HK":"Hong Kong","ID":'Indonesia'}

Target_cities= {'US':
                      ['New York', 'Chicago', 'San Francisco', 'Austin', 'Seattle',
                  'Los Angeles', 'Philadelphia', 'Atlanta', 'Dallas',
                  'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston','Miami'],
                'SG':["Singapore"],
                'MY':['Kuala Lumpur','Johor Bahru','Shah Alam'],
                'HK':['Hong Kong'],
                'ID':['Jakarta','Batam','Surabaya']
                }

In [6]:
URL = {"SG":"https://www.indeed.com.sg/jobs",
       "US":"https://www.indeed.com/jobs",
       "MY":"https://www.indeed.com.my/jobs",
       "HK":"https://www.indeed.hk/jobs",
       "ID":"https://id.indeed.com/lowongan-kerja"
       
       }

In [55]:
max_results_per_city = 20000
#put data scientist as a placeholder, will be filled with the list of jobs later
parameters = {'q': 'data scientist', 'radius': '100', 'start':1}
#list of jobs
jobs = ['data scientist','data analyst']

In [22]:
def scrape_page_to_df(url, url_params,country):
    """
    extract information from a results page and save to an existing csv
    :param url: url template
    :param url_params: a dictionary to feed to params argument in requests.get (based on the parameters I defined above, and I'll make a wrapper to do this below)
    :return: a pandas dataframe containing the extracted information
    """
    # create a empty dictionary to store extracted information
    scraped_data = {'location': [],
                  'company': [],
                  'title': [],
                  'salary': [],
                  'description': [],
                  'review': [],
                  'star': [],
                  'country':[]
                  }

    html = requests.get(url, params=url_params)

    # make sure the response status is ok
    assert html.status_code == requests.codes.ok

    soup = BeautifulSoup(html.text, 'lxml')

  #helper function to extract results

    def extract_results(soup):
        return soup.find_all('div', class_='result')

    results = extract_results(soup)

  #helper functions to extract information
    def extract_location(result):
        """extract job location"""
        try:
            location = result.find('span', class_='location').get_text().strip()
            return location
        except:
            return None


    def extract_company(result):
        """extract the name of the company"""
        try:
            company = result.find('span', class_='company').get_text().strip()
            return company
        except:
            return None

    def extract_title(result):
        """extract the job title"""
        try:
            title = result.find('a', attrs={'data-tn-element': "jobTitle"}).get('title')
            return title
        except:
            return None


    def extract_salary(result):
        """extract the salary"""
        try:
            salary = result.find('td', class_='snip').\
            find('span', class_='no-wrap').\
            get_text().strip()
            return salary
        except:
            return None


    def extract_description(result):
        """extract job description snippet"""
        try:
            description = result.find('span', class_='summary').get_text().strip()
            return description
        except:
            return None


    def extract_review(result):
        """extract the number of reviews for the company"""
        try:
            review = result.find('a', attrs={'data-tn-element': "reviewStars"})
            review = review.find('span', class_="slNoUnderline")
            review = review.get_text().strip()
            # extract only the number
            review = review.replace(',', '').replace(' reviews', '')
            return review
        except:
            return None            

    
    def extract_star(result):
        """extract a number (width) that is proportional to the number of stars
        shown for the company"""
        try:
            # the 'style' attribute dictates how many stars are filled with color
            star = result.find('span', class_='rating').get('style')
            # extract only the number
            star = star.replace('width:', '').replace('px', '')
            return star
        except:
            return None


    # append extracted info to the correspond list
    for result in results:
        scraped_data['location'].append(extract_location(result))
        scraped_data['company'].append(extract_company(result))
        scraped_data['title'].append(extract_title(result))
        scraped_data['salary'].append(extract_salary(result))
        scraped_data['description'].append(extract_description(result))
        scraped_data['review'].append(extract_review(result))
        scraped_data['star'].append(extract_star(result))
        scraped_data['country'].append(country)

      # convert the dictionary to a pandas dataframe and returns it
    return pd.DataFrame(scraped_data)


helper function to remove duplicates

In [23]:
def remove_duplicates(df):
    """remove duplicates and returns a new df"""
    
    nrows_before = df.shape[0]
    df.drop_duplicates(subset=['company', 'country','description',
                               'location', 'salary', 'title'],
                       keep='last', inplace=True)
    nrows_after = df.shape[0]
    
    print('{} rows remain after removing duplicates from {} rows.'.format(
        nrows_after, nrows_before))
    print('{} rows have salary info; {} rows have yearly salary info.'.format(
      df.salary.notnull().sum(), df.salary.str.contains('year').sum()))
    return df

In [24]:
def scrapper(CountryCode):
    print('Current system time: {}'.format(time.ctime()))
    
   
  
    # scrape data and save to dataframe
    start_time = time.time()
    
    #Retrieve Parameters to scrape with based on input of Country Code
    url = URL[CountryCode]
    locations = Target_cities[CountryCode]
    country = Countries[CountryCode]
    
    #Create an empty place holder df, search through every location in that country, but only 1 results, just to get the title and columns
    df = scrape_page_to_df(url,parameters,country)
    
    
    for tempat in locations:
        for kerja in jobs:
            for start in range(0, max_results_per_city, 10):
            
              
                url_params = parameters.copy()
                #update the job with the target job that we want, city for target city that we are looking for and start refers to the current page number being scrapped
                url_params.update({'l': tempat,'q': kerja, 'start': start})


                #insert code to put the scrap stuff into a df here, after each round of loop, concat into a df
                df = pd.concat([df,scrape_page_to_df(url, url_params,country)],axis=0)
              
        print('Finished scraping {}'.format(tempat))
    total_time = (time.time() - start_time) / 60
    print('Scraping run time: {:.1f} minutes'.format(total_time))

    
    
    
    # remove duplicates
    df = remove_duplicates(df)
    print('Script finished at {}\n'.format(time.ctime()))
    
    #returns the final df
    return df

In [ ]:
SG = scrapper('SG')

In [ ]:
SG.to_pickle('./data/SG.pkl')
SG.to_csv('./data/SG.csv')

In [26]:
ID = scrapper('ID')

Current system time: Mon Oct 23 00:45:35 2017
Finished scraping Jakarta
Finished scraping Batam
Finished scraping Surabaya
Scraping run time: 54.9 minutes
1968 rows remain after removing duplicates from 49459 rows.
33 rows have salary info; 0 rows have yearly salary info.
Script finished at Mon Oct 23 01:40:30 2017



In [52]:
ID.to_pickle('./data/ID.pkl')
ID.to_csv('./data/ID.csv')

In [43]:
US = scrapper('US')

Current system time: Mon Oct 23 10:33:01 2017
Finished scraping New York
Finished scraping Chicago
Finished scraping San Francisco
Finished scraping Austin
Finished scraping Seattle
Finished scraping Los Angeles
Finished scraping Philadelphia
Finished scraping Atlanta
Finished scraping Dallas
Finished scraping Pittsburgh
Finished scraping Portland
Finished scraping Phoenix
Finished scraping Denver
Finished scraping Houston
Finished scraping Miami
Scraping run time: 411.0 minutes
17395 rows remain after removing duplicates from 151302 rows.
1271 rows have salary info; 848 rows have yearly salary info.
Script finished at Mon Oct 23 17:23:59 2017



In [51]:
US.to_pickle('./data/US.pkl')
US.to_csv('./data/US.csv')

In [56]:
HK= scrapper('HK')

Current system time: Mon Oct 23 19:48:32 2017
Finished scraping Hong Kong
Scraping run time: 29.2 minutes
899 rows remain after removing duplicates from 39999 rows.
12 rows have salary info; 0 rows have yearly salary info.
Script finished at Mon Oct 23 20:17:47 2017



In [ ]:
HK.to_pickle('./data/HK.pkl')
HK.to_csv('./data/HK.csv')

In [ ]:
MY=scrapper("MY")


Current system time: Mon Oct 23 20:17:47 2017
Finished scraping Kuala Lumpur


In [ ]:
MY.to_pickle('./data/MY.pkl')
MY.to_csv('./data/MY.csv')

In [54]:
alldf =[SG,MY,ID,US,HK]

Should pickle these

In [35]:
import pickle 

In [36]:
pickle.load('./data/SG.pkl')

TypeError: file must have 'read' and 'readline' attributes